In [2]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import folium
import requests
from shapely.geometry import LineString
import polyline
from tqdm import tqdm


closest_cities = gpd.read_feather("closest_cities.feather")
test_point = Point(40.627537, 47.589869)  # (lon, lat)

In [3]:
def flip_geometry(geom):
    if isinstance(geom, LineString):
        # Reverse the coordinates for LineString
        return LineString([(y, x) for x, y in geom.coords])

    return geom  # Return unchanged for other geometry types


def get_route(start_coords, end_coords):
    """
    Чтобы подружить эту всю историю с Transport frames (TF), нужно просто заменить этот метод.
    Что конкретно нужно заменить, так это то, как получается маршрут.
    Тут он просто стучится в сторонний АПИ и получает ответ.
    Чтобы это было через осм граф (коим и оперирует TF), нужно где-то засторить дорожный граф на всю рашку.
    Есть идея сторить его где-то не весь, а в делении по АДМ например. И через условно sjoin получать нужный кусок где находится точка (тут нужно все же учесть уровень на котором в TF хранится граф).
    Если условно по изменению графа в TF его сохранять с новым айдишником, запускать джобу по подгрузке его куда-то сюда,
    то при запросе сюда можно также передавать айди нужного графа. Тогда можно условно сравнить до-после.

    Пути (всм на машине) через osmnx можно получить, там не сложно.
    Либо такую штуку как это локально поднять по туториалу от девелоперов osrm.
    """

    CONST_SEC_IN_H = 3600
    # print(start_coords, end_coords)
    # Construct the URL for the OSRM route API
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[0]},{start_coords[1]};{end_coords[0]},{end_coords[1]}?overview=full"

    # Send the GET request to the OSRM API
    response = requests.get(url)

    # Check if the response is successful
    if response.status_code == 200:
        route_data = response.json()

        # Check if the route was found
        if route_data["code"] == "Ok":
            # Decode the geometry from polyline format
            geometry = route_data["routes"][0]["geometry"]
            decoded_points = polyline.decode(geometry)
            line = LineString(decoded_points)
            line = flip_geometry(line)
            return {
                "distance": route_data["routes"][0]["distance"],
                "duration": round(
                    route_data["routes"][0]["duration"] / CONST_SEC_IN_H, 2
                ),
                "geometry": line,  # List of (lat, lng) tuples
            }
        else:
            print("Error in routing:", route_data["message"])
            return None
    else:
        print("HTTP Error:", response.status_code)
        return None


get_routes = lambda city: get_route((test_point.x, test_point.y), (city.x, city.y))

data = list(
    tqdm(map(get_routes, closest_cities["geometry"]), total=len(closest_cities))
)
# Создание GeoDataFrame
gdf = gpd.GeoDataFrame(data)

# Установка CRS, если необходимо
gdf.set_crs(epsg=4326, inplace=True)
closest_cities.loc[:, "hours"] = gdf["duration"].values

100%|██████████| 9/9 [00:04<00:00,  2.18it/s]


In [4]:
closest_cities.to_feather("closest_cities.feather")

In [5]:
# Создание базовой карты
m = folium.Map(
    location=[test_point.y, test_point.x], zoom_start=9, tiles="Cartodb dark_matter"
)

# Добавление начальной точки на карту с зеленым цветом
folium.Marker(
    location=[test_point.y, test_point.x],
    popup="Start Point",
    icon=folium.Icon(color="green"),  # Установка цвета маркера
).add_to(m)
# Добавление первого GeoDataFrame на карту
folium.GeoJson(
    gdf, style_function=lambda x: {"color": "red", "weight": 2, "fillOpacity": 0.5}
).add_to(m)

# Добавление второго GeoDataFrame на карту
folium.GeoJson(
    closest_cities,
    style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.5},
).add_to(m)

# Отображение карты
m